In [2]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Simulated user-item interaction matrix (rows = users, columns = items)
# 1 means liked/swapped
interactions = np.array([
    [1, 0, 1, 0],  # User 0
    [0, 1, 1, 0],  # User 1
    [1, 1, 0, 1],  # User 2
])

item_names = ["Red Hoodie", "White Shirt", "Blue Jeans", "Summer Dress"]
item_descriptions = [
    "Casual red hoodie for winter wear",
    "Formal white cotton shirt",
    "Denim blue jeans, regular fit",
    "Floral summer dress with light fabric"
]

print("=== Collaborative Filtering Recommendations for User 0 ===")
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(interactions)

# Find nearest users to user 0
user_id = 0
_, neighbors = model.kneighbors([interactions[user_id]], n_neighbors=3)
neighbor_ids = neighbors[0][1:]  # exclude self

# Recommend items liked by neighbors but not by user 0
recommended = set()
for neighbor in neighbor_ids:
    for idx, liked in enumerate(interactions[neighbor]):
        if liked == 1 and interactions[user_id][idx] == 0:
            recommended.add(item_names[idx])

print(f"Recommended from similar users: {list(recommended)}")

print("\n=== Content-Based Filtering: Items Similar to 'Red Hoodie' ===")
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(item_descriptions)

cos_sim = (tfidf_matrix * tfidf_matrix[0].T).toarray()
top_indices = np.argsort(cos_sim[:, 0])[::-1][1:3]

print("Items similar to 'Red Hoodie':")
for i in top_indices:
    print("-", item_names[i])

print("\n=== Text-Based Recommendation: Query 'I like hoodies and winter wear' ===")
query = "I like hoodies and winter wear"
embedder = SentenceTransformer('all-MiniLM-L6-v2')

query_vec = embedder.encode(query, convert_to_tensor=True)
item_vecs = embedder.encode(item_descriptions, convert_to_tensor=True)

cos_scores = util.pytorch_cos_sim(query_vec, item_vecs)[0]
top_query_idx = cos_scores.argsort(descending=True)[:3]

print("Top matches for query:")
for i in top_query_idx:
    print(f"- {item_names[i]} (score: {cos_scores[i]:.2f})")


=== Collaborative Filtering Recommendations for User 0 ===
Recommended from similar users: ['White Shirt', 'Summer Dress']

=== Content-Based Filtering: Items Similar to 'Red Hoodie' ===
Items similar to 'Red Hoodie':
- Summer Dress
- Blue Jeans

=== Text-Based Recommendation: Query 'I like hoodies and winter wear' ===
Top matches for query:
- Red Hoodie (score: 0.65)
- Blue Jeans (score: 0.37)
- Summer Dress (score: 0.36)
